# Getting started

A [Stream](https://thermosteam.readthedocs.io/en/latest/Stream.html) object is the main interface for estimating thermodynamic properties, vapor-liquid equilibrium, and material and energy balances. 

### Material and energy balance

Before creating a stream, first define both the working chemicals using a [Chemicals](https://thermosteam.readthedocs.io/en/latest/Chemicals.html) object and the thermodynamic property package through a [Thermo](https://thermosteam.readthedocs.io/en/latest/Thermo.html) object:

In [1]:
import thermosteam as tmo
thermo = tmo.Thermo(['Water', 'Ethanol'])
thermo

Thermo(chemicals=CompiledChemicals([Water, Ethanol]), mixture=Mixture('ideal mixing rules', ..., rigorous_energy_balance=True, include_excess_energies=False), Gamma=DortmundActivityCoefficients, Phi=IdealFugacityCoefficients, PCF=IdealPoyintingCorrectionFactors)

Now we can set the thermo property package and start creating streams:

In [2]:
tmo.settings.set_thermo(thermo)
s1 = tmo.Stream('s1', Water=20, Ethanol=20, units='kg/hr')
s1.show(flow='kg/hr')

Stream: s1
 phase: 'l', T: 298.15 K, P: 101325 Pa
 flow (kg/hr): Water    20
               Ethanol  20


Create another stream at a higher temperature:

In [3]:
s2 = tmo.Stream('s2', Water=10, units='kg/hr', T=350, P=101325)
s2.show(flow='kg/hr')

Stream: s2
 phase: 'l', T: 350 K, P: 101325 Pa
 flow (kg/hr): Water  10


Mix both stream into a new one:

In [4]:
s_mix = tmo.Stream('s_mix')
s_mix.mix_from([s1, s2])
s_mix.show(flow='kg/hr')

Stream: s_mix
 phase: 'l', T: 310.53 K, P: 101325 Pa
 flow (kg/hr): Water    30
               Ethanol  20


Check the energy balance through enthalpy:

In [5]:
s_mix.H - (s1.H + s2.H)

0.001944559560797643

Note that the balance is not perfect as the solver stops within a small temperature tolerance. However, the approximation is less than 0.1% off:

In [6]:
error = s_mix.H - (s1.H + s2.H)
percent_error = 100 * error / (s1.H + s2.H)
print(f"{percent_error:.2%}")

0.01%


Split the mixture to two streams by defining the component splits:

In [7]:
# First define an array of component splits
component_splits = s_mix.chemicals.array(['Water', 'Ethanol'], [0, 1])
s_mix.split_to(s1, s2, component_splits)
s1.T = s2.T = s_mix.T # Take care of energy balance
s1.show(flow='kg/hr')
s2.show(flow='kg/hr')

Stream: s1
 phase: 'l', T: 310.53 K, P: 101325 Pa
 flow (kg/hr): Ethanol  20
Stream: s2
 phase: 'l', T: 310.53 K, P: 101325 Pa
 flow (kg/hr): Water  30


### Flow rates

The most convinient way to get and set flow rates is through the `get_flow` and `set_flow` methods:

In [8]:
# Set and get flow of a single chemical
# in gallons per minute
s1.set_flow(1, 'gpm', 'Water')
s1.get_flow('gpm', 'Water')

1.0

In [9]:
# Set and get flows of many chemicals
# in kilograms per hour
s1.set_flow([10, 20], 'kg/hr', ('Ethanol', 'Water'))
s1.get_flow('kg/hr', ('Ethanol', 'Water'))

array([10., 20.])

It is also possible to index flow rate data using chemical IDs through the `imol`, `imass`, and `ivol` [indexers](https://thermosteam.readthedocs.io/en/latest/indexer/indexer_module.html):

In [10]:
s1.imol.show()

ChemicalMolarFlowIndexer (kmol/hr):
 (l) Water    1.11
     Ethanol  0.2171


In [11]:
s1.imol['Water']

1.1101687012358397

In [12]:
s1.imol['Ethanol', 'Water']

array([0.217, 1.11 ])

All flow rates are stored as an array in the `mol` attribute:

In [13]:
s1.mol # Molar flow rates [kmol/hr]

array([1.11 , 0.217])

Mass and volumetric flow rates are available as [property arrays](https://free-properties.readthedocs.io/en/latest/property_array.html):

In [14]:
s1.mass

property_array([<Water: 20 kg/hr>, <Ethanol: 10 kg/hr>])

In [15]:
s1.vol

property_array([<Water: 0.019846 m^3/hr>, <Ethanol: 0.012909 m^3/hr>])

These arrays work just like ordinary arrays, but the data is linked to the molar flows:

In [16]:
# Mass flows are always up to date with molar flows
s1.mol[0] = 1
s1.mass[0]

<Water: 18.015 kg/hr>

In [17]:
# Changing mass flows changes molar flows
s1.mass[0] *= 2
s1.mol[0]

2.0

In [18]:
# Property arrays act just like normal arrays
s1.mass + 2 # A new array is created

array([38.031, 12.   ])

In [19]:
# Array methods are also the same
s1.mass.mean()

23.01528

### Thermal condition

Temperature and pressure can be get and set through the `T` and `P` attributes:

In [20]:
s1.T = 400.
s1.P = 2 * 101325.
s1.show()

Stream: s1
 phase: 'l', T: 400 K, P: 202650 Pa
 flow (kmol/hr): Water    2
                 Ethanol  0.217


The phase may also be changed ('s' for solid, 'l' for liquid, and 'g' for gas):

In [21]:
s1.phase = 'g'

Notice that VLE is not enforced, but it is possible to perform. For now, just check that the dew point is lower than the actual temperature to assert it must be gas:

In [22]:
dp = s1.dew_point_at_P() # Dew point at constant pressure
dp

DewPointValues(T=390.8422316772931, P=202650.0, IDs=('Water', 'Ethanol'), z=[0.902 0.098], x=[0.991 0.009])

In [23]:
dp.T < s1.T

True

It is also possible to get and set in other units of measure:

In [24]:
s1.set_property('P', 1, 'atm')
s1.get_property('P', 'atm')

1.0

In [25]:
s1.set_property('T', 125, 'degC')
s1.get_property('T', 'degF')

257.0000004

Enthalpy can also be set. An energy balance is made to solve for temperature at isobaric conditions:

In [26]:
s1.H = s1.H + 500
s1.get_property('T', 'degC') # Temperature should go up

130.80212615289332

### Thermal properties

Thermodynamic properties are pressure, temperature and phase dependent. In the following examples, let's just use water as it is easier to check properties:

In [27]:
s_water = tmo.Stream('s_water', Water=1, units='kg/hr')
s_water.rho # Density [kg/m^3]

1002.4679023140482

In [28]:
s_water.T = 350
s_water.rho # Density changes

1028.8572592234138

Get properties in different units:

In [29]:
s_water.get_property('sigma', 'N/m') # Surface tension

0.06329591766859191

In [30]:
s_water.get_property('V', 'm3/kmol') # Molar volume

0.017509989688557978

### Flow properties

Several flow properties are available, such as net material and energy flow rates:

In [31]:
# Net molar flow rate [kmol/hr]
s_water.F_mol

0.05550843506179199

In [32]:
# Net mass flow rate [kg/hr]
s_water.F_mass

1.0

In [33]:
# Net volumetric flow rate [m3/hr]
s_water.F_vol

0.0009719521255599678

In [34]:
# Enthalpy flow rate [kJ/hr]
s_water.H

216.85380295250482

In [35]:
# Entropy flow rate [kJ/hr]
s_water.S

0.670540696937784

In [36]:
# Capacity flow rate [J/K]
s_water.C

4.197679245159573

### Vapor-liquid equilibrium

Vapor-liquid equilibrium can be performed by setting 2 degrees of freedom from the following list: `T` (Temperature; in K), `P` (Pressure; in Pa), `V` (Vapor fraction), and `H` (Enthalpy; in kJ/hr).

For example, set vapor fraction and pressure:

In [37]:
s_eq = tmo.Stream('s_eq', Water=10, Ethanol=10)
s_eq.vle(V=0.5, P=101325)
s_eq.show(composition=True)

MultiStream: s_eq
 phases: ('g', 'l'), T: 353.88 K, P: 101325 Pa
 composition: (g) Water    0.3861
                  Ethanol  0.6139
                  -------  10 kmol/hr
              (l) Water    0.6138
                  Ethanol  0.3862
                  -------  10 kmol/hr


Note that the stream is a now a MultiStream to manage multiple phases. Each phase can be accessed separately too:

In [38]:
s_eq['l'].show()

Stream: 
 phase: 'l', T: 353.88 K, P: 101325 Pa
 flow (kmol/hr): Water    6.14
                 Ethanol  3.86


In [39]:
s_eq['g'].show()

Stream: 
 phase: 'g', T: 353.88 K, P: 101325 Pa
 flow (kmol/hr): Water    3.86
                 Ethanol  6.14


Note that the phase of these substreams cannot be changed:

In [40]:
s_eq['g'].phase = 'l'

AttributeError: phase is locked

Again, the most convinient way to get and set flow rates in is through the `get_flow` and `set_flow` methods:

In [41]:
# Set flow of liquid water
s_eq.set_flow(1, 'gpm', ('l', 'Water'))
s_eq.get_flow('gpm', ('l', 'Water'))

1.0

In [42]:
# Set multiple liquid flows
key = ('l', ('Ethanol', 'Water'))
s_eq.set_flow([10, 20], 'kg/hr', key)
s_eq.get_flow('kg/hr', key)

array([10., 20.])

Chemical flows across all phases can be retrieved if no phase is given:

In [43]:
# Get water and ethanol flows summed across all phases
s_eq.get_flow('kg/hr', ('Water', 'Ethanol'))

array([ 89.563, 292.783])

However, setting chemical data of MultiStream objects requires the phase to be specified:

In [44]:
s_eq.set_flow([10, 20], 'kg/hr', ('Water', 'Ethanol'))

IndexError: multiple phases present; must include phase key to set chemical data

Similar to Stream objects, all flow rates can be accessed through the `imol`, `imass`, and `ivol` attributes:

In [45]:
s_eq.imol # Molar flow rates

MolarFlowIndexer (kmol/hr):
 (g) Water     3.861
     Ethanol   6.138
 (l) Water     1.11
     Ethanol   0.2171


In [46]:
# Index a single chemical in the liquid phase
s_eq.imol['l', 'Water']

1.1101687012358397

In [47]:
# Index multiple chemicals in the liquid phase
s_eq.imol['l', ('Ethanol', 'Water')]

array([0.217, 1.11 ])

In [48]:
# Index the vapor phase
s_eq.imol['g']

array([3.861, 6.138])

In [49]:
# Index flow of chemicals summed across all phases
s_eq.imol['Ethanol', 'Water']

array([6.355, 4.971])

Because multiple phases are present, overall chemical flows in MultiStream objects cannot be set like in Stream objects:

In [50]:
s_eq.imol['Ethanol', 'Water'] = [1, 0]

IndexError: multiple phases present; must include phase key to set chemical data

Chemical flows must be set by phase:

In [51]:
s_eq.imol['l', ('Ethanol', 'Water')] = [1, 0]

One main difference between a [MultiStream](https://thermosteam.readthedocs.io/en/latest/MultiStream.html) object and a [Stream](https://thermosteam.readthedocs.io/en/latest/Stream.html) object is that the `mol` attribute no longer stores any data, it simply returns the total flow rate of each chemical. Setting an element of the array raises an error to prevent the wrong assumption that the data is linked:

In [52]:
s_eq.mol

array([3.861, 7.138])

In [53]:
s_eq.mol[0] = 1

ValueError: assignment destination is read-only

Note that for both Stream and MultiStream objects, `get_flow`, `imol`, and `mol` return chemical flows across all phases when given only chemical IDs.

### Liquid-liquid equilibrium

Liquid-liquid equilibrium (LLE) only requires the temperature. Pressure is not a significant variable as liquid fungacity coefficients are not a strong function of pressure. 

In [54]:
tmo.settings.set_thermo(['Water', 'Butanol', 'Octane'])
liquid_mixture = tmo.Stream('liquid_mixture', Water=100, Octane=100, Butanol=5)
liquid_mixture.lle(T=300)
liquid_mixture

MultiStream: liquid_mixture
 phases: ('L', 'l'), T: 300 K, P: 101325 Pa
 flow (kmol/hr): (L) Water    98.54
                     Butanol  1.209
                     Octane   0.001977
                 (l) Water    1.458
                     Butanol  3.791
                     Octane   100


Compared to VLE, LLE is an order of magnitude times slower. This is because differential evolution, a purely stochastic method, is used to find the solution that globally minimizes the gibb's free energy of both phases.